# Lyrics Analysis

In [1]:
lyrics_directory = "Lyrics_Data/"
tracks_csv = "tracks_group.csv"
stopwords_file = "stopwords.txt"

In [2]:
target_group = "seventeen"
kpop_group = set(['kpop', 'itzy', 'seventeen', 'bts', 'twice'])
global_group = set(['global', 'billboard'])

target_and_kpop = kpop_group.union(set([target_group]))
target_and_global = global_group.union(set([target_group]))

## Import data

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(tracks_csv)

df.head(3)

,track_name,track_id,grouping
0,Ice Cream (with Selena Gomez),2J4P46vCFm1rPkNkp9pZWX,kpop
1,Dynamite,0v1x6rN6JHRapa03JElljE,kpop
2,THE BADDEST,2V4Fx72svQRxrFvNT1eq5f,kpop


In [5]:
df.shape

(4618, 3)

In [6]:
df["grouping"].unique()

array(['kpop', 'itzy', 'seventeen', 'bts', 'twice', 'global', 'billboard'],
      dtype=object)

In [7]:
def get_lyrics(track_id):
    try:
        lyrics = ""
        with open(lyrics_directory + track_id + ".txt", "r") as f:
            lyrics = f.read()
            
        return lyrics            
    except:
        return None
    
df["lyrics"] = df["track_id"].apply(get_lyrics)

df.shape

(4618, 4)

In [8]:
# stop words
# https://www.kaggle.com/datasets/rowhitswami/stopwords/
stopwords = set()
with open(stopwords_file, "r") as f:
    stopwords = set(f.read().split("\n"))
    
stopwords

{',',
 '>',
 '?',
 'Just',
 'Yes',
 'a',
 'abaft',
 'abafter',
 'abaftest',
 'about',
 'abouter',
 'aboutest',
 'above',
 'abover',
 'abovest',
 'accordingly',
 'aer',
 'aest',
 'afore',
 'after',
 'afterer',
 'afterest',
 'afterward',
 'afterwards',
 'again',
 'against',
 'aid',
 'ain',
 'albeit',
 'all',
 'aller',
 'allest',
 'alls',
 'allyou',
 'almost',
 'along',
 'alongside',
 'already',
 'also',
 'although',
 'always',
 'amid',
 'amidst',
 'among',
 'amongst',
 'an',
 'and',
 'andor',
 'anear',
 'anent',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anywhere',
 'apart',
 'aparter',
 'apartest',
 'appear',
 'appeared',
 'appearing',
 'appears',
 'appropriate',
 'appropriated',
 'appropriater',
 'appropriates',
 'appropriatest',
 'appropriating',
 'are',
 'ares',
 'around',
 'as',
 'ases',
 'aside',
 'asides',
 'aslant',
 'astraddle',
 'astraddler',
 'astraddlest',
 'astride',
 'astrider',
 'astridest',
 'at',
 'athwart',
 'atop',
 'atween',
 'aught',
 'aught

## Analyze

In [9]:
def is_stopword(word):
    word = word.lower()
    
    if len(word) <= 1:
        return True
    
    # in the list
    if word in stopwords:
        return True
    
    # is Korean
    if ord("가") <= ord(word[0]) <= ord("힣"):
        return True
    
    return False

### Characteristic Keyword
Using TF-IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### Among global

In [11]:
df_looking = df[df["grouping"].isin(target_and_global)].reset_index(drop=True)

In [12]:
vectorizer = TfidfVectorizer(stop_words=list(stopwords), max_features=1000, min_df=10)

x = vectorizer.fit_transform(df_looking["lyrics"].fillna("")).toarray()

df_tfidf = pd.DataFrame(x, columns=vectorizer.get_feature_names_out()).T

c:\Users\Hiyoko\.conda\envs\data\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['vis', 'viser', 'visest', 'yes'] not in stop_words.
  warnings.warn(


In [13]:
threshold = 0.001
counter = {}
summer = {}

for cnt in range(len(df_looking)):
    if df_looking["grouping"][cnt] != target_group:
        continue
    
    tfidf_series = df_tfidf[cnt]
    
    keywords = tfidf_series[tfidf_series > threshold].index.tolist()
    for keyword in keywords:
        counter[keyword] = counter.get(keyword, 0) + 1
        summer[keyword] = summer.get(keyword, 0) + float(tfidf_series[keyword])

In [14]:
def sort_dictionary(dictionary):
    return [(k, dictionary[k]) for k in sorted(dictionary, key=dictionary.get, reverse=True)]

In [15]:
tfidf_counter_global = sort_dictionary(counter)
tfidf_counter_global

[('home', 81),
 ('sorry', 80),
 ('news', 74),
 ('let', 72),
 ('heart', 61),
 ('love', 60),
 ('think', 48),
 ('baby', 42),
 ('new', 39),
 ('night', 39),
 ('only', 39),
 ('follow', 38),
 ('feel', 38),
 ('world', 38),
 ('end', 34),
 ('eyes', 34),
 ('back', 33),
 ('contact', 33),
 ('keep', 32),
 ('yes', 31),
 ('say', 31),
 ('looking', 30),
 ('leave', 30),
 ('dream', 30),
 ('mind', 29),
 ('happy', 29),
 ('sleep', 28),
 ('hands', 27),
 ('need', 25),
 ('yeh', 25),
 ('give', 25),
 ('morning', 25),
 ('find', 24),
 ('take', 24),
 ('am', 24),
 ('hand', 24),
 ('sky', 24),
 ('wait', 23),
 ('worry', 23),
 ('smile', 22),
 ('room', 21),
 ('tell', 21),
 ('ah', 20),
 ('sun', 20),
 ('dance', 20),
 ('meet', 20),
 ('today', 19),
 ('life', 19),
 ('music', 19),
 ('away', 18),
 ('way', 18),
 ('play', 18),
 ('light', 18),
 ('hear', 18),
 ('wind', 18),
 ('little', 17),
 ('change', 17),
 ('still', 17),
 ('child', 17),
 ('easy', 17),
 ('tomorrow', 17),
 ('place', 17),
 ('remember', 17),
 ('share', 16),
 ('call', 

In [16]:
tfidf_summer_global = sort_dictionary(summer)
tfidf_summer_global

[('home', 15.736966679808186),
 ('news', 14.518524685914194),
 ('sorry', 11.91577797479948),
 ('heart', 8.811270094419466),
 ('love', 8.688800538611188),
 ('baby', 7.788746881942375),
 ('contact', 6.832942029185571),
 ('yeh', 6.542506996848126),
 ('world', 6.220133391429427),
 ('let', 6.0279484609790055),
 ('follow', 5.612129997871336),
 ('dream', 5.371939112373928),
 ('only', 5.096528380955112),
 ('new', 5.071695046562155),
 ('end', 4.9540229578548205),
 ('think', 4.712041506435446),
 ('happy', 4.496023423359318),
 ('back', 4.005746254043052),
 ('give', 3.7335500790827494),
 ('looking', 3.6827297732633477),
 ('wind', 3.670400931134151),
 ('worry', 3.6425433783146657),
 ('smile', 3.589368585436858),
 ('shut', 3.571071355579521),
 ('leave', 3.5368330698902164),
 ('ay', 3.4872861784621976),
 ('night', 3.483549123653218),
 ('yes', 3.44780622637595),
 ('run', 3.4310940206210545),
 ('song', 3.418346823020923),
 ('life', 3.364894648261042),
 ('um', 3.3641824991900555),
 ('music', 3.347872862

#### Among KPOP

In [17]:
df_looking = df[df["grouping"].isin(target_and_kpop)].reset_index(drop=True)

In [18]:
vectorizer = TfidfVectorizer(stop_words=list(stopwords), max_features=1000, min_df=10)

x = vectorizer.fit_transform(df_looking["lyrics"].fillna("")).toarray()

df_tfidf = pd.DataFrame(x, columns=vectorizer.get_feature_names_out()).T

c:\Users\Hiyoko\.conda\envs\data\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['vis', 'viser', 'visest', 'yes'] not in stop_words.
  warnings.warn(


In [19]:
threshold = 0.001
counter = {}
summer = {}

for cnt in range(len(df_looking)):
    if df_looking["grouping"][cnt] != target_group:
        continue
    
    tfidf_series = df_tfidf[cnt]
    
    keywords = tfidf_series[tfidf_series > threshold].index.tolist()
    for keyword in keywords:
        counter[keyword] = counter.get(keyword, 0) + 1
        summer[keyword] = summer.get(keyword, 0) + float(tfidf_series[keyword])

In [20]:
tfidf_counter_kpop = sort_dictionary(counter)
tfidf_counter_kpop

[('home', 80),
 ('sorry', 80),
 ('news', 74),
 ('let', 72),
 ('heart', 61),
 ('love', 60),
 ('think', 48),
 ('baby', 42),
 ('night', 40),
 ('only', 40),
 ('new', 39),
 ('follow', 38),
 ('feel', 38),
 ('world', 38),
 ('end', 34),
 ('eyes', 34),
 ('back', 33),
 ('contact', 33),
 ('keep', 32),
 ('yes', 31),
 ('say', 31),
 ('looking', 30),
 ('leave', 30),
 ('dream', 30),
 ('mind', 29),
 ('happy', 29),
 ('sleep', 28),
 ('hands', 27),
 ('need', 25),
 ('yeh', 25),
 ('give', 25),
 ('morning', 25),
 ('find', 24),
 ('take', 24),
 ('am', 24),
 ('hand', 24),
 ('sky', 24),
 ('wait', 23),
 ('worry', 23),
 ('tell', 22),
 ('smile', 22),
 ('room', 21),
 ('ah', 20),
 ('sun', 20),
 ('dance', 20),
 ('comment', 20),
 ('meet', 20),
 ('today', 19),
 ('life', 19),
 ('music', 19),
 ('away', 18),
 ('price_varies', 18),
 ('way', 18),
 ('play', 18),
 ('light', 18),
 ('hear', 18),
 ('wind', 18),
 ('little', 17),
 ('change', 17),
 ('still', 17),
 ('child', 17),
 ('easy', 17),
 ('song', 17),
 ('questions', 17),
 ('t

In [21]:
tfidf_summer_kpop = sort_dictionary(summer)
tfidf_summer_kpop

[('home', 12.15820554016299),
 ('love', 9.029950266202746),
 ('heart', 7.8499695759031205),
 ('baby', 7.383031344719721),
 ('news', 7.112408166336561),
 ('sorry', 6.890558195969502),
 ('only', 5.749848884568152),
 ('yeh', 5.713494252203872),
 ('let', 5.712994427725181),
 ('think', 5.069674096544697),
 ('world', 4.969372652295935),
 ('back', 4.683235562496428),
 ('give', 4.351780180473527),
 ('end', 4.047276678189264),
 ('dream', 4.043967495151701),
 ('new', 4.031170116408636),
 ('happy', 4.013403552645574),
 ('contact', 3.9751484423575043),
 ('life', 3.706154365069785),
 ('run', 3.6648673415278665),
 ('follow', 3.6406844515529726),
 ('looking', 3.619639156326462),
 ('yes', 3.5779200142749383),
 ('night', 3.481554551838795),
 ('say', 3.4765591544016083),
 ('feel', 3.3922187540080064),
 ('song', 3.3899439264015068),
 ('call', 3.38277431976287),
 ('am', 3.371554583814393),
 ('music', 3.370028153834897),
 ('find', 3.316858735067583),
 ('leave', 3.3125686232572074),
 ('ay', 3.24221179522855

### Topic analysis

In [22]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

In [23]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
def save_lda_vis(lda, corpus, dictionary, filename):
    vis = gensimvis.prepare(lda, corpus, dictionary)
    pyLDAvis.save_html(vis, filename)
    
    with open(filename, "r") as f:
        html = f.read()
        
    return html, vis

In [24]:
def pre_process(df_lyrics):
    # tokenize lyrics
    lyrics_processed = df_lyrics.fillna("").str.split()

    for _list in lyrics_processed:
        for cnt in range(len(_list)):
            _list[cnt] = _list[cnt].replace(".", "").replace(",", "").replace("?", "").replace("\\", "").replace("/", "").replace(":", "").lower()
            
            if is_stopword(_list[cnt]):
                _list[cnt] = ""
                
    # remove empty string
    for cnt in range(len(lyrics_processed)):
        while(True):
            try:
                lyrics_processed[cnt].remove("")
            except:
                break

    return lyrics_processed

#### Among global

In [25]:
df_looking = df[df["grouping"].isin(target_and_global)].reset_index(drop=True)

In [26]:
#train model
lyrics_processed = pre_process(df_looking["lyrics"])
dictionary = Dictionary(lyrics_processed)
corpus = [dictionary.doc2bow(text) for text in lyrics_processed]
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=100)

all topics

In [27]:
lda_html_global, vis = save_lda_vis(lda, corpus, dictionary, "lda.html")

vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
41    -0.075941  0.025363       1        1  5.236018
60    -0.109383 -0.018190       2        1  4.039108
50    -0.074735  0.037664       3        1  3.618778
58    -0.054359  0.012946       4        1  3.473303
3     -0.103541 -0.006483       5        1  2.553072
...         ...       ...     ...      ...       ...
5      0.141167  0.054347      96        1  0.248734
22     0.123255  0.004245      97        1  0.243588
91     0.072147  0.043320      98        1  0.240190
23     0.078366 -0.116384      99        1  0.237605
33     0.099912  0.052669     100        1  0.182397

[100 rows x 5 columns], topic_info=         Term         Freq        Total  Category  logprob  loglift
4827   labour  5764.000000  5764.000000   Default  30.0000  30.0000
55       baby  5261.000000  5261.000000   Default  29.0000  29.0000
229     child  2244.000000  2244.000000 

what topics are in kpop?

In [28]:
threshold = 0.3

counter = {}
summer = {}

for cnt in range(len(df_looking)):
    if df_looking["grouping"][cnt] != target_group:
        continue
    
    topics = lda.get_document_topics(corpus[cnt], minimum_probability=0)
    
    for topic in topics:
        if topic[1] > threshold:
            counter[topic[0]] = counter.get(topic[0], 0) + 1
            
        summer[topic[0]] = summer.get(topic[0], 0) + float(topic[1])

In [29]:
_lda_counter_global = sort_dictionary(counter)
lda_counter_global = []
for tup in _lda_counter_global:
    new_tup = [tup[0], tup[1], lda.show_topic(tup[0])]
    lda_counter_global.append(new_tup)
    
lda_counter_global

[[84,
  10,
  [('singing', 0.05771063),
   ('eyes', 0.048666075),
   ('love', 0.019763183),
   ('back', 0.017304605),
   ("let's", 0.012228269),
   ('only', 0.011277615),
   ('air', 0.0100965435),
   ('world', 0.009427535),
   ('summer', 0.009081703),
   ('home', 0.008101674)]],
 [60,
  9,
  [('love', 0.13947676),
   ('tell', 0.034091163),
   ('give', 0.014979607),
   ('baby', 0.01335559),
   ('think', 0.012266691),
   ('let', 0.01084674),
   ('girl', 0.0103097465),
   ('yes', 0.009375715),
   ("can't", 0.009330926),
   ('crazy', 0.009017908)]],
 [44,
  7,
  [('shake', 0.068554066),
   ('news', 0.037835088),
   ('say', 0.0124783255),
   ('heart', 0.011804123),
   ("that's", 0.011529324),
   ("can't", 0.010363023),
   ('hate', 0.010337144),
   ('love', 0.01016198),
   ('break', 0.009900742),
   ('play', 0.009600614)]],
 [41,
  7,
  [('life', 0.03154448),
   ('love', 0.015483348),
   ('feel', 0.013643192),
   ('down', 0.012519144),
   ('take', 0.010631998),
   ('need', 0.008462441),
   (

In [30]:
_lda_summer_global = sort_dictionary(summer)
lda_summer_global = []
for tup in _lda_summer_global:
    new_tup = [tup[0], tup[1], lda.show_topic(tup[0])]
    lda_summer_global.append(new_tup)
    
lda_summer_global

[[44,
  10.886646459070107,
  [('shake', 0.068554066),
   ('news', 0.037835088),
   ('say', 0.0124783255),
   ('heart', 0.011804123),
   ("that's", 0.011529324),
   ("can't", 0.010363023),
   ('hate', 0.010337144),
   ('love', 0.01016198),
   ('break', 0.009900742),
   ('play', 0.009600614)]],
 [84,
  10.816426437459086,
  [('singing', 0.05771063),
   ('eyes', 0.048666075),
   ('love', 0.019763183),
   ('back', 0.017304605),
   ("let's", 0.012228269),
   ('only', 0.011277615),
   ('air', 0.0100965435),
   ('world', 0.009427535),
   ('summer', 0.009081703),
   ('home', 0.008101674)]],
 [60,
  7.243056149105541,
  [('love', 0.13947676),
   ('tell', 0.034091163),
   ('give', 0.014979607),
   ('baby', 0.01335559),
   ('think', 0.012266691),
   ('let', 0.01084674),
   ('girl', 0.0103097465),
   ('yes', 0.009375715),
   ("can't", 0.009330926),
   ('crazy', 0.009017908)]],
 [24,
  6.913185040859389,
  [('hope', 0.027218571),
   ('lonely', 0.026719578),
   ('call', 0.018332059),
   ('alone', 0

#### Among KPOP

In [31]:
df_looking = df[df["grouping"].isin(target_and_kpop)].reset_index(drop=True)

In [32]:
#train model
lyrics_processed = pre_process(df_looking["lyrics"])
dictionary = Dictionary(lyrics_processed)
corpus = [dictionary.doc2bow(text) for text in lyrics_processed]
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=100)

# observe topics
topics = lda.print_topics()

In [33]:
threshold = 0.3

counter = {}
summer = {}

for cnt in range(len(df_looking)):
    if df_looking["grouping"][cnt] != target_group:
        continue
    
    topics = lda.get_document_topics(corpus[cnt], minimum_probability=0)
    
    for topic in topics:
        if topic[1] > threshold:
            counter[topic[0]] = counter.get(topic[0], 0) + 1
            
        summer[topic[0]] = summer.get(topic[0], 0) + float(topic[1])


In [34]:
lda_html_kpop, vis = save_lda_vis(lda, corpus, dictionary, "lda.html")

vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
98    -0.013261  0.319958       1        1  3.728599
75    -0.035179  0.029836       2        1  3.425587
47    -0.019564 -0.005821       3        1  2.030040
31     0.314772  0.022695       4        1  1.934781
20    -0.029576 -0.088611       5        1  1.932774
...         ...       ...     ...      ...       ...
39    -0.002388  0.135822      96        1  0.381017
56    -0.004654 -0.041385      97        1  0.346362
73    -0.030063 -0.003335      98        1  0.339607
9      0.263529  0.000660      99        1  0.295549
53    -0.024848  0.004406     100        1  0.234322

[100 rows x 5 columns], topic_info=         Term         Freq        Total  Category  logprob  loglift
280     child  3837.000000  3837.000000   Default  30.0000  30.0000
425     drunk  2516.000000  2516.000000   Default  29.0000  29.0000
494      cake  1478.000000  1478.000000 

In [35]:
_lda_counter_kpop = sort_dictionary(counter)
lda_counter_kpop = []
for tup in _lda_counter_kpop:
    new_tup = [tup[0], tup[1], lda.show_topic(tup[0])]
    lda_counter_kpop.append(new_tup)
    
lda_counter_kpop

[[20,
  11,
  [('home', 0.18265785),
   ('love', 0.017707422),
   ('sorry', 0.011809134),
   ('shut', 0.011513994),
   ('am', 0.009352348),
   ('on!', 0.009270272),
   ("let's", 0.008456439),
   ('news', 0.008390028),
   ('comment', 0.0082049165),
   ('attention', 0.007047505)]],
 [43,
  10,
  [('drunk', 0.21904062),
   ('home', 0.047671597),
   ('aite', 0.018595224),
   ('news', 0.010326509),
   ("can't", 0.009181527),
   ('night', 0.008046023),
   ('follow', 0.0075025833),
   ('love', 0.007339297),
   ('run', 0.0072843586),
   ('leave', 0.007255459)]],
 [36,
  7,
  [('dimensioni', 0.047060277),
   ('́â', 0.026219964),
   ('ya', 0.021853546),
   ('ay', 0.021774983),
   ('kim', 0.020240657),
   ('cuz', 0.016768552),
   ('feel', 0.013074756),
   ('room', 0.011701567),
   ('music', 0.011697477),
   ('aya', 0.011682937)]],
 [0,
  6,
  [('singing', 0.20280877),
   ('sì', 0.032113086),
   ('hot', 0.027350131),
   ('home', 0.023200016),
   ('only', 0.022777732),
   ('best', 0.018557042),
   

In [36]:
_lda_summer_kpop = sort_dictionary(summer)
lda_summer_kpop = []
for tup in _lda_summer_kpop:
    new_tup = [tup[0], tup[1], lda.show_topic(tup[0])]
    lda_summer_kpop.append(new_tup)
    
lda_summer_kpop

[[20,
  11.908442973213823,
  [('home', 0.18265785),
   ('love', 0.017707422),
   ('sorry', 0.011809134),
   ('shut', 0.011513994),
   ('am', 0.009352348),
   ('on!', 0.009270272),
   ("let's", 0.008456439),
   ('news', 0.008390028),
   ('comment', 0.0082049165),
   ('attention', 0.007047505)]],
 [43,
  9.013091972639813,
  [('drunk', 0.21904062),
   ('home', 0.047671597),
   ('aite', 0.018595224),
   ('news', 0.010326509),
   ("can't", 0.009181527),
   ('night', 0.008046023),
   ('follow', 0.0075025833),
   ('love', 0.007339297),
   ('run', 0.0072843586),
   ('leave', 0.007255459)]],
 [36,
  7.568412618107686,
  [('dimensioni', 0.047060277),
   ('́â', 0.026219964),
   ('ya', 0.021853546),
   ('ay', 0.021774983),
   ('kim', 0.020240657),
   ('cuz', 0.016768552),
   ('feel', 0.013074756),
   ('room', 0.011701567),
   ('music', 0.011697477),
   ('aya', 0.011682937)]],
 [1,
  6.262613752955076,
  [('am', 0.02252264),
   ('baby', 0.014921169),
   ('moon', 0.012561385),
   ('cherry', 0.0121

### Emotion analysis
Emotions:  
fear
anger
anticipation
trust
surprise
positive
negative
sadness
disgust
joy

In [37]:
import numpy as np

In [38]:
labels = [
    "anger",
    "anticipation",
    "disgust",
    "fear",
    "joy",
    "sadness",
    "surprise",	
    "trust",
    "negative",	
    "positive",
]

emotions = [
    "anger",
    "anticipation",
    "disgust",
    "fear",
    "joy",
    "sadness",
    "surprise",	
    "trust",
]

sentiments = [
    "negative",	
    "positive",
]

In [39]:
df_nrc = pd.read_csv("NRC.txt", sep="\t", names=["word", "label", "amount"])

words_to_labels = {}

for _, row in df_nrc.iterrows():
    words_to_labels[row["word"]] = words_to_labels.get(row["word"], np.zeros((10, 1)))
    words_to_labels[row["word"]][labels.index(row["label"])] = row["amount"]

In [40]:
df_label = df.copy()
for label in labels:
    df_label[label] = 0

In [41]:
for cnt in range(len(df)):
    lyrics = df["lyrics"][cnt]

    if type(lyrics) != str:
        continue

    label_this = np.zeros((10, 1))

    for word in lyrics.split():
        if word in words_to_labels:
            label_this += words_to_labels[word]
            
    emotion_this = label_this[:8]
    sentiment_this = label_this[8:]
            
    if np.linalg.norm(emotion_this) != 0:
        emotion_this /= np.linalg.norm(emotion_this)
        
    if np.linalg.norm(sentiment_this) != 0:
        sentiment_this /= np.linalg.norm(sentiment_this)
        
    label_this = np.concatenate((emotion_this, sentiment_this), axis=None)
    
    for label in labels:
        df_label[label][cnt] = label_this[labels.index(label)]

C:\Users\Hiyoko\AppData\Local\Temp\ipykernel_14252\3885107406.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_label[label][cnt] = label_this[labels.index(label)]
C:\Users\Hiyoko\AppData\Local\Temp\ipykernel_14252\3885107406.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.0810361063877192' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_label[label][cnt] = label_this[labels.index(label)]
C:\Users\Hiyoko\AppData\Local\Temp\ipykernel_14252\3885107406.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [42]:
df_emotion_global = df_label[df_label["grouping"].isin(global_group)]
mean_emotion_global = df_emotion_global[emotions].mean()
normalized_mean_emotion_global = mean_emotion_global / np.linalg.norm(mean_emotion_global)
mean_sentiment_global = df_emotion_global[sentiments].mean()
normalized_mean_sentiment_global = mean_sentiment_global / np.linalg.norm(mean_sentiment_global)

df_emotion_kpop = df_label[df_label["grouping"].isin(kpop_group)]
mean_emotion_kpop = df_emotion_kpop[emotions].mean()
normalized_mean_emotion_kpop = mean_emotion_kpop / np.linalg.norm(mean_emotion_kpop)
mean_sentiment_kpop = df_emotion_kpop[sentiments].mean()
normalized_mean_sentiment_kpop = mean_sentiment_kpop / np.linalg.norm(mean_sentiment_kpop)

df_emotion_target = df_label[df_label["grouping"] == target_group]
mean_emotion_target = df_emotion_target[emotions].mean()
normalized_mean_emotion_target = mean_emotion_target / np.linalg.norm(mean_emotion_target)
mean_sentiment_target = df_emotion_target[sentiments].mean()
normalized_mean_sentiment_target = mean_sentiment_target / np.linalg.norm(mean_sentiment_target)

In [43]:
normalized_mean_emotion_target

anger           0.142321
anticipation    0.524079
disgust         0.111218
fear            0.274743
joy             0.547596
sadness         0.277381
surprise        0.212057
trust           0.442112
dtype: float64

In [44]:
normalized_mean_sentiment_target

negative    0.449481
positive    0.893290
dtype: float64

against global

In [45]:
emotion_percentage_global = (normalized_mean_emotion_target - normalized_mean_emotion_global) / normalized_mean_emotion_global * 100

emotion_percentage_global

anger          -52.714827
anticipation    27.215481
disgust        -42.776791
fear           -17.133726
joy             10.694337
sadness        -24.500793
surprise       -10.374876
trust           11.823479
dtype: float64

In [46]:
sentiment_percentage_global = (normalized_mean_sentiment_target - normalized_mean_sentiment_global) / normalized_mean_sentiment_global * 100

sentiment_percentage_global

negative   -31.464169
positive    18.331409
dtype: float64

against KPOP

In [47]:
emotion_percentage_kpop = (normalized_mean_emotion_target - normalized_mean_emotion_kpop) / normalized_mean_emotion_kpop * 100

emotion_percentage_kpop

anger          -35.911127
anticipation    14.959537
disgust        -20.309264
fear            -7.054993
joy             -4.826208
sadness        -12.253606
surprise       -10.188302
trust           14.427958
dtype: float64

In [48]:
sentiment_percentage_kpop = (normalized_mean_sentiment_target - normalized_mean_sentiment_kpop) / normalized_mean_sentiment_kpop * 100

sentiment_percentage_kpop

negative   -11.847695
positive     3.842093
dtype: float64

## Save Data

In [49]:
# to float
for _sum in lda_summer_global:
    _sum[2] = [(tup[0], float(tup[1])) for tup in _sum[2]]
    
for _sum in lda_summer_kpop:
    _sum[2] = [(tup[0], float(tup[1])) for tup in _sum[2]]
    
for count in lda_counter_global:
    count[2] = [(tup[0], float(tup[1])) for tup in count[2]]

for count in lda_counter_kpop:
    count[2] = [(tup[0], float(tup[1])) for tup in count[2]]

In [50]:
json_data = {
    "tfidf_counter_global": tfidf_counter_global,
    "tfidf_summer_global": tfidf_summer_global,
    "tfidf_counter_kpop": tfidf_counter_kpop,
    "tfidf_summer_kpop": tfidf_summer_kpop,
    "lda_counter_global": lda_counter_global,
    "lda_summer_global": lda_summer_global,
    "lda_counter_kpop": lda_counter_kpop,
    "lda_summer_kpop": lda_summer_kpop,
    "lda_html_global": lda_html_global,
    "lda_html_kpop": lda_html_kpop,
    "normalized_mean_emotion_target": normalized_mean_emotion_target.astype(float).tolist(),
    "sentiment_percentage_global": sentiment_percentage_global.astype(float).tolist(),
    "emotion_percentage_global": emotion_percentage_global.astype(float).tolist(),
    "sentiment_percentage_global": sentiment_percentage_global.astype(float).tolist(),
    "emotion_percentage_kpop": emotion_percentage_kpop.astype(float).tolist(),
    "sentiment_percentage_kpop": sentiment_percentage_kpop.astype(float).tolist(),
}

In [51]:
import json
with open("data_"+ target_group +".json", "w") as f:
    json.dump(json_data, f)